In [1]:
%reload_ext autoreload
%autoreload 2

import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as trans

import torch
import os
import sys

sys.path.append('../')
sys.path.append('code/')

from phase2_finetune_CRL import finetune_CRL
from phase2_finetune_baseline import finetune_baseline
from THINGS_load_data import load_all

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.10 (you have 1.4.8). Upgrade using: pip install --upgrade albumentations


In [2]:
plt.rc('figure', dpi=100, figsize=(15,8))
plt.rc('font', size=12, weight=500)
plt.rc('lines', linewidth=2)
plt.rc('axes', facecolor='FAFAFA', edgecolor='C3C3C3', axisbelow=True, grid=True)

#### GOD dataset ROIs

In [3]:
# load sbj_1.npz

data = np.load('data/Kamitani/npz/sbj_1.npz')
print(data.files)

['V1', 'V2', 'V3', 'V4', 'FFA', 'PPA', 'LOC', 'LVC', 'HVC', 'VC', 'arr_0', 'arr_1', 'arr_2', 'arr_3', 'arr_4', 'arr_5', 'arr_6']


In [4]:
data['arr_0'].shape, data['arr_1'].shape, data['arr_2'].shape, data['arr_3'].shape, data['arr_4'].shape, data['arr_5'].shape, data['arr_6'].shape

((1200, 4466), (1750, 4466), (50, 4466), (1200,), (1750,), (4466,), (4466,))

In [5]:
for roi in data.files[:10]:
    print(roi, np.sum(data[roi], axis=0))

V1 1004
V2 1018
V3 759
V4 740
FFA 568
PPA 356
LOC 540
LVC 2281
HVC 2049
VC 4466


#### THINGS dataset ROIs

In [ ]:
selected_ROI = ["glasser-V1", "glasser-V2", "glasser-V3", "glasser-V4", "glasser-LO1", "glasser-LO2", "glasser-LO3", "glasser-VMV1", "glasser-VMV2", "glasser-VMV3", "glasser-PHA1", "glasser-PHA2", "glasser-PHA3"]

full_sets, train_sets, test_sets, labels = load_all(selectedROI=selected_ROI, reload_resnet=True, ratio=0.6)
responses, voxdata, stimdata, images, image_features, semantic_emb, similarity_emb, full_categories = full_sets
res_train, im_train, imf_train, sem_train, simi_train, cat_train = train_sets
res_test, im_test, imf_test, sem_test, simi_test, cat_test = test_sets

for i in range(13):
    print(selected_ROI[i], np.sum(voxdata[voxdata.columns[i+1]]))

print("Data shape:")
print(res_train.shape, im_train.shape, res_test.shape, im_test.shape)

glasser-V1 2211
glasser-V2 1817
glasser-V3 1167
glasser-V4 667
glasser-LO1 133
glasser-LO2 129
glasser-LO3 418
glasser-VMV1 168
glasser-VMV2 371
glasser-VMV3 99
glasser-PHA1 145
glasser-PHA2 208
glasser-PHA3 135


### Autoencoder joint training (phase 2)

#### GOD

In [154]:
### Training of autoencoders with 
#   - no interaction (independent AEs)
#   - cross-attention (bidirectional residual stream flow)

class config_up():
    def __init__(self):
        self.pretrain_path = 'checkpoints/checkpoints_pre_140_doublecontra.pth'
        self.fmri_recon_weight = 0.25
        self.img_recon_weight = 1.5
        self.img_mask_ratio = 0.5
        self.mask_ratio = 0.75
        self.batch_size = 4
        self.dataset = 'GOD'
        self.num_epoch = 40

        self.seed = 0
        self.do_cross_attention = True

config_update = config_up()

metrics_path = 'results/tests'

for do_cross_attention in [True, False]:
    for random_seed in [0, 1, 2, 3, 4]:
        config_update.do_cross_attention = do_cross_attention
        config_update.seed = random_seed
        train_metrics, eval_metrics, saved_models = finetune_baseline(config_update)

        save_name = str(('cross' if do_cross_attention else 'bare') + '_seed' + str(random_seed) + '_standard' + '.pkl')
        with open(os.path.join(metrics_path, save_name), 'wb') as f:
            to_save = {'train_metrics': train_metrics, 'eval_metrics': eval_metrics, 'saved_models': saved_models}
            pickle.dump(to_save, f)

In [11]:
### Training of autoencoders with causal module (CRL)

class config_up():
    def __init__(self):
        self.pretrain_path = 'checkpoints/checkpoints_pre_140_doublecontra.pth'
        self.fmri_recon_weight = 0.25
        self.img_recon_weight = 1.75
        self.img_mask_ratio = 0.5
        self.mask_ratio = 0.75
        self.batch_size = 16
        self.dataset = 'GOD'
        self.num_epoch = 40

        self.fmri_encoding_dim = 1024
        self.img_encoding_dim = 768
        self.debug = False
        self.seed = 0

config_update = config_up()

model_params = {
    'num_layer': [1,1],
    'num_layer_z': 1,
    'num_neurons_hidden': 75,
    'directions': np.array([[0, 1],
                        [1, 0]]),
    'phi_type': 'gauss-maxout'
}
train_params = {
    'initial_learning_rate': 0.03,
    'weight_decay': 0.,
}

metrics_path = 'results/tests'

for phi_type in ['gauss-relu', 'gauss-mlp', 'gauss-maxout']:
    for random_seed in [0, 1, 2, 3, 4]:
        model_params['phi_type'] = phi_type
        config_update.seed = random_seed
        train_metrics, eval_metrics, saved_models = finetune_CRL(config_update, model_params, train_params)

        save_name = str(model_params['phi_type']).split('-')[1] + '_seed' + str(config_update.seed) + '_standard' + '.pkl'
        with open(os.path.join(metrics_path, save_name), 'wb') as f:
            to_save = {'train_metrics': train_metrics, 'eval_metrics': eval_metrics, 'saved_models': saved_models}
            pickle.dump(to_save, f)

In [6]:
# clear cuda cache and memory

torch.cuda.empty_cache()
torch.cuda.memory.empty_cache()

#### THINGS

In [ ]:
### Training of autoencoders with 
#   - no interaction (independent AEs)
#   - cross-attention (bidirectional residual stream flow)

class config_up():
    def __init__(self):
        self.pretrain_path = 'checkpoints/checkpoints_pre_140_doublecontra.pth'
        #self.pretrain_path = 'results/fmri_finetune_THINGS_sbj_1/02-07-2024-10-23-20/checkpoints_39/checkpoint_singlesub_cross_att_GOD_s_fmriw1.0_imgw1.0_fmar0.75_imar0.75_epo39_mergconf.pth'
        self.fmri_recon_weight = 0.25
        self.img_recon_weight = 1.5
        self.img_mask_ratio = 0.5
        self.mask_ratio = 0.75
        self.batch_size = 16
        self.dataset = 'THINGS'
        self.num_epoch = 40

        self.seed = 0
        self.do_cross_attention = True

config_update = config_up()

metrics_path = 'results/tests/things'

for do_cross_attention in [True, False]:
    for random_seed in [0, 1, 2, 3, 4]:
        config_update.do_cross_attention = do_cross_attention
        config_update.seed = random_seed
        train_metrics, eval_metrics, saved_models = finetune_baseline(config_update)

        save_name = 'bare' + '_seed' + str(random_seed) + '_things' + '.pkl'
        with open(os.path.join(metrics_path, save_name), 'wb') as f:
            to_save = {'train_metrics': train_metrics, 'eval_metrics': eval_metrics, 'saved_models': saved_models}
            pickle.dump(to_save, f)

[63662, 81034, 81034, 81034]
cuda:1
Batch size: 16
Number of epochs: 30


/home/aip/dufly/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/aip/dufly/anaconda3/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Cross attention: True
Number of voxels: 4656
Cross residual True
Loaded model from /home/aip/dufly/Documents/GOD/checkpoints/checkpoints_pre_140_doublecontra.pth , number of voxels 4656
Dataset: THINGS


It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Original train_set.fmri.shape: (6912, 4592)
Original test_set.fmri.shape: (60, 4592)
New train_set.fmri.shape: (6912, 4656)
New test_set.fmri.shape: (60, 4656)
Dataset size: 6912, 60
Finetuning MAE on train fMRI ... ...
[Epoch 0] Train loss/corr fMRI: 1.100 | 0.050
           Train loss/corr image: 0.039 | 0.663
           Train loss TOTAL: 0.334
[Test 0] Test loss/corr fMRI: 0.978 | 0.118
           Test loss/corr image: 2.524e-04 | 0.719
Epoch end, total time: 1:15
[Epoch 1] Train loss/corr fMRI: 0.946 | 0.131
           Train loss/corr image: 2.370e-04 | 0.699
           Train loss TOTAL: 0.237
[Test 1] Test loss/corr fMRI: 0.947 | 0.167
           Test loss/corr image: 1.312e-04 | 0.641
Epoch end, total time: 2:24
Saving models in file: ./results/fmri_finetune_THINGS_sbj_1/31-07-2024-10-55-37
Saving model at ep 1 eval_image_corr_train 0.13132228847179148 eval_image_corr_test 0.1667064093053341, best ep 1,  best eval_loss 0.9740406796336174
[Epoch 2] Train loss/corr fMRI: 0.924 | 0.

In [ ]:
### Training of autoencoders with causal module (CRL)

class config_up():
    def __init__(self):
        self.pretrain_path = 'checkpoints/checkpoints_pre_140_doublecontra.pth'
        #self.pretrain_path = 'results/fmri_finetune_THINGS_sbj_1/02-07-2024-10-23-20/checkpoints_39/checkpoint_singlesub_cross_att_GOD_s_fmriw1.0_imgw1.0_fmar0.75_imar0.75_epo39_mergconf.pth'
        self.fmri_recon_weight = 0.25
        self.img_recon_weight = 1.75
        self.img_mask_ratio = 0.5
        self.mask_ratio = 0.75
        self.batch_size = 16
        self.dataset = 'THINGS'
        self.num_epoch = 40

        self.fmri_encoding_dim = 1024
        self.img_encoding_dim = 768
        self.debug = False
        self.seed = -1

config_update = config_up()

model_params = {
    'num_layer': [0, 0],
    'num_layer_z': 1,
    'num_neurons_hidden': 75,
    'directions': np.array([[0, 1],
                        [1, 0]]),
    'phi_type': 'gauss-maxout'
}
train_params = {
    'initial_learning_rate': 0.03,
    'weight_decay': 0.,
}

metrics_path = 'results/tests/things'

for recon_mult in [0.5, 1, 2, 4]:
    for random_seed in [0, 1, 2, 3, 4]:
        config_update.fmri_recon_weight = recon_mult * 0.25
        config_update.img_recon_weight = recon_mult * 1.75
        config_update.seed = random_seed
        train_metrics, eval_metrics, saved_models = finetune_CRL(config_update, model_params, train_params)

        save_name = str(model_params['phi_type']).split('-')[1] + '_seed' + str(config_update.seed) + 'standard.pkl'
        with open(os.path.join(metrics_path, save_name), 'wb') as f:
            to_save = {'train_metrics': train_metrics, 'eval_metrics': eval_metrics, 'saved_models': saved_models}
            pickle.dump(to_save, f)

[26524, 6988, 13404, 22612, 22390, 13406, 13534, 13534]
cuda:0
Batch size: 16
Number of epochs: 40
Patch size: 16
Loaded model from /home/aip/dufly/Documents/GOD/checkpoints/checkpoints_pre_140_doublecontra.pth , number of voxels 4656
Dataset: THINGS
Original train_set.fmri.shape: (6912, 4592)
Original test_set.fmri.shape: (1728, 4592)
New train_set.fmri.shape: (6912, 4656)
New test_set.fmri.shape: (1728, 4656)
Dataset size: 6912, 1728
Initialized CRL framework
Finetuning MAE on train fMRI ... ...
[Epoch 0] Train loss/corr fMRI: 1.169 | 0.490
           Train loss/corr image: 0.039 | 0.662
           Train loss/accu/lr CRL: 1.059 | 0.547|2.158e-05
           Train loss TOTAL: 1.420
[Test 0] Test loss/corr fMRI: 0.973 | 0.541
           Test loss/corr image: 2.490e-04 | 0.679
           Test loss/accu CRL: 1.815 | 0.541
Epoch end, total time: 5:59
[Epoch 1] Train loss/corr fMRI: 0.956 | 0.547
           Train loss/corr image: 2.331e-04 | 0.702
           Train loss/accu/lr CRL: 2.183 | 

FileNotFoundError: [Errno 2] No such file or directory: '/home/aip/dufly/Documents/GOD/results/tests/things/maxout_seed0standard.pkl'